In [30]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB #朴素贝叶斯模块
from sklearn.datasets import load_digits #导入数据集
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC #支持向量机的分类
from sklearn.ensemble import RandomForestClassifier as RFC #随机森林
from sklearn.tree import DecisionTreeClassifier as DTC#决策树
from sklearn.linear_model import LogisticRegression as LR #逻辑回归
from sklearn.model_selection import learning_curve#学习曲线
from sklearn.model_selection import ShuffleSplit #随机交叉验证
from time import time
import datetime

In [1]:
import pandas as pd

In [15]:
#parse_dates=['Dates']保留datatime的属性
train = pd.read_csv(r'Kaggle\Kaggle\train.csv',parse_dates=['Dates'])

In [16]:
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
Dates         878049 non-null datetime64[ns]
Category      878049 non-null object
Descript      878049 non-null object
DayOfWeek     878049 non-null object
PdDistrict    878049 non-null object
Resolution    878049 non-null object
Address       878049 non-null object
X             878049 non-null float64
Y             878049 non-null float64
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 60.3+ MB


In [18]:
test =pd.read_csv(r'Kaggle\Kaggle\test.csv',parse_dates=['Dates'],index_col=0)

In [19]:
test.head()

,Dates,DayOfWeek,PdDistrict,Address,X,Y
Id,,,,,,
0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884262 entries, 0 to 884261
Data columns (total 6 columns):
Dates         884262 non-null datetime64[ns]
DayOfWeek     884262 non-null object
PdDistrict    884262 non-null object
Address       884262 non-null object
X             884262 non-null float64
Y             884262 non-null float64
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 47.2+ MB


In [21]:
train.shape

(878049, 9)

In [22]:
test.shape

(884262, 6)

In [29]:
len(train['Category'].unique())

39

In [24]:
len(train.loc[:,'Category'].value_counts())

39

In [33]:
from sklearn.preprocessing import LabelEncoder #标签的编码化，就是给标签一个编码

In [36]:
leCrime = LabelEncoder()
crime = leCrime.fit_transform(train.Category)
crime

array([37, 21, 21, ..., 16, 35, 12])

因子化 pd.get_dummies

In [37]:
#get_dummies 因子化,就是取值变为列，做一个升维，原始数据中对应的标成1，这里是DayOfWeek这一列
days = pd.get_dummies(train.DayOfWeek)
days.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [38]:
#分类型变量因子化，数据必须是分类型，其他数据可进行分箱后在因子化
district = pd.get_dummies(train.PdDistrict)
district.head()

,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0


In [40]:
#筛选小时
hour = train.Dates.dt.hour
hour = pd.get_dummies(hour)
hour.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [45]:
traindata=pd.concat([hour,days,district],axis=1)
traindata.head()

,0,1,2,3,4,5,6,7,8,9,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [46]:
traindata['crime']=crime#标签

In [55]:
#建模
#切分数据
Xtrain,Xtest,Ytrain,Ytest = train_test_split(traindata.iloc[:,:-1],
                                             traindata.iloc[:,-1],
                                             test_size=0.2)

In [49]:
from sklearn.preprocessing import MinMaxScaler #归一化
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.preprocessing import KBinsDiscretizer 
from sklearn.naive_bayes import BernoulliNB

In [56]:
#伯努利 
bnb = BernoulliNB()
bnb.fit(Xtrain,Ytrain)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [57]:
#计算损失函数，计算真实值和预测值的偏差
proba = bnb.predict_proba(Xtest) #计算预测值

In [58]:
from sklearn.metrics import log_loss #模型评估

In [61]:
logloss = log_loss(Ytest,proba)
logloss

2.5886584071266823

In [60]:
test.shape

(884262, 6)

test是真正的测试值，上面只是进行了一个拟合，所有多训练集做的操作也要对测试集做一遍

In [63]:
days = pd.get_dummies(test.DayOfWeek)
district = pd.get_dummies(test.PdDistrict)
hour = test.Dates.dt.hour
hour = pd.get_dummies(hour)
testdata = pd.concat([hour,days,district],axis=1)
testdata.head()

,0,1,2,3,4,5,6,7,8,9,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
Id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [64]:
#进行标签的预测
bnb.predict(testdata)

array([21, 21, 16, ..., 21, 21, 21])